# 计算文本相似度

## 读取文件

In [7]:
from vsm_similarity import read_file, cal_tf_idf, cal_doc_norm, cal_similarity, cal_similarity_mp
file_name = 'sample.txt'
docs, words = read_file(file_name)

这一部分耗时2秒以内。

读取完毕后，`docs`中保存着文档的预处理信息，其中每个元素都是一篇文档，以`docs[0]`为例：

```json
{
    "id": "01-01-001",
    "max_tf": 24,
    "length": 626,
    "words": ["经验/n", "永远/d", ...],
    "content": "迈向充满希望的新世纪——一九九八年新年讲话..."
} 
```

`words`则保存着所有词语在不同文档中的词频，形如：

```json
{
  "迈向/v": {
    0: 2,
    2: 1,
    36: 1,
    ...,
    2969: 1
  },
  "充满/v": {
    0: 3,
    1: 3,
    2: 2,
    ...,
    3125: 1
  },
  ...
}
```


## 预计算

In [8]:
tf_idfs = cal_tf_idf(docs, words)
cal_doc_norm(docs, tf_idfs)

这一部分耗时1秒以内。

计算所有词语在不同文档中的tf-idf，并保存在相应的数据结构中，形如：
```json
{
  "迈向/v": {
    0: 0.40297056538909104,
    2: 0.4835646784669093,
    36: 0.6044558480836366,
    ...,
    2969: 0.11794260450412422
  },
  "充满/v": {
    0: 0.43890347840841126,
    ...
  },
  ...
}
```

然后由得到的tf-idf计算各个文档向量的模长，保存在`docs`的新字段中。

## 计算相似度

In [9]:
sim_mp = cal_similarity_mp(docs, tf_idfs, pnum=4)

In [10]:
sim = cal_similarity(docs, tf_idfs)

这里提供了两种计算方式，单进程和多进程。其中单进程耗时35秒以上，并行数4的多进程耗时25秒以内。

In [11]:
import math
all([math.isclose(a, b) for a, b in zip(sim, sim_mp)])

True

可以验证两种计算方式得到的结果是一致的。

## 验证结果

In [12]:
def get_simililarity(i, j):
    """获得文档i和文档j的相似度"""
    assert len(docs) > j
    assert j > i
    return sim[j-i + (2*len(docs) - i - 1)*i//2 - 1]


def get_specific(i):
    """获得特定文档i与其他文档的相似度"""
    assert len(docs) > i
    ret = []
    for j in range(i):
        ret.append(get_simililarity(j, i))
    ret.append(1)
    index = ((2*len(docs) - i - 1)*i)//2
    ret += sim[index:index+len(docs)-i-1]
    return ret


def most_similar(i, n=20):
    """查找与文档i最相似的文档"""
    v = get_specific(i)
    return sorted(((j, v[j]) for j in range(len(docs))), key=lambda x: x[1], reverse=True)[:n]

这里定义了一些实用函数，方便后续的结果验证，可自行任意调用。

In [22]:
import random

target = random.randint(0, len(docs)-1)
res = most_similar(target)
print(docs[target]['content'])      # 目标文档的内容
print()                             # 换个行
print(docs[res[1][0]]['content'])   # 最相似的文档的内容
print(res)                          # 具体相似度值

温家宝在贵州农村考察时指出动员全社会力量打好扶贫攻坚战
新华社贵阳１２月３１日电（记者刘子富）中共中央政治局委员、中央书记处书记温家宝日前在贵州农村考察扶贫工作时指出，到本世纪末基本解决农村贫困人口的温饱问题，是党的十五大提出的经济社会发展总体部署的一项重要任务，有重大和深远的意义。各级领导干部要从全局和战略的高度认识和对待扶贫工作，切实加强对扶贫工作的领导，坚持开发扶贫的方针，加大扶贫攻坚力度，动员全社会力量，打好扶贫攻坚战。
１２月２７日至３１日，温家宝在贵州省委书记刘方仁等陪同下，在黔东南苗族侗族自治州、黔南布依族苗族自治州冒雨考察扶贫工作，看望各族干部群众，转达党中央、国务院的亲切关怀和问候。
近年来，贵州省各级党委和政府把扶贫开发工作作为农村中心任务来抓，取得很大成绩，贫困人口大量减少，贫困状况明显改善。温家宝在农民家中详细询问他们生产生活情况，同干部群众一起研究扶贫开发的路子。他说，必须坚持开发扶贫的方针，通过发展经济解决贫困人口的温饱问题。要把农业生产尤其是粮食生产放在第一位，首先解决群众吃饭问题。同时，面向市场需求，充分利用当地资源，积极发展多种经营，增加农民收入。温家宝考察了农田水利建设工地，他说，要大搞农田基本建设，植树造林，治水改土，改善生产条件和生态环境，使脱贫建立在比较坚实的物质技术基础之上。
温家宝在黔南州民族干部学校，与师生亲切交谈。他说，贫困地区要改变贫困面貌，必须从根本上改变教育、科技、文化落后的状况，通过义务教育、成人教育等多种形式提高群众的文化水平。积极推广各种实用技术，把增产增效显著，易于掌握的技术送给农民，用于生产实践，增强脱贫致富的能力。
温家宝一家又一家地看望了贫困户，与各族群众促膝交谈。他说，扶贫工作进入攻坚阶段，任务十分艰巨。贫困地区的广大干部要落实责任，加大工作力度，切实把扶贫工作做深、做细，做到村、做到户，逐村逐户落实扶贫政策措施，确定生产开发项目，搞好科技信息服务。有扶贫任务的地方、部门和单位，要发扬扶贫济困的优良传统，努力搞好对口扶贫。贫困地区的广大群众要坚定信心，坚持自力更生、艰苦创业，用自己的双手改变贫困面貌。
温家宝与当地乡、村各族干部进行了座谈，他勉励干部要牢记党的全心全意为人民服务的宗旨，密切联系群众，关心群众生活，深入基层，帮助群众解决实际困难和问题。元旦、春节到了，特别要安排好灾区和贫困地区群众